In [2]:
import MetaTrader5 as mt
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
from datetime import datetime, timedelta
import time

In [3]:
mt.initialize()


True

In [4]:
login = 
password = #Add your login credentials
server = 
mt.login(login, password, server)

True

In [5]:
#load our trained model
model = load_model('C:/Users/Admin/Documents/ML bot/saved model/keras extension/US30 1m.keras')
ticker = "US30"
buy = mt.ORDER_TYPE_BUY
sell = mt.ORDER_TYPE_SELL

#this function creates or places orders at market price
def create_order(ticker, qty, order_type):
    request = {
        "action": mt.TRADE_ACTION_DEAL,
        "symbol": ticker,
        "volume": qty,
        "type": order_type,
        #"sl": sl,
        #"tp": tp,
        "comment": "Position Opened",
        "type_time": mt.ORDER_TIME_GTC,
        "type_filling": mt.ORDER_FILLING_IOC,
    }
    create_order = mt.order_send(request)
    return create_order

#this fn closes existing orders
def close_order(ticker, qty, ticket, order_type):
    request = {
        "action": mt.TRADE_ACTION_DEAL,
        "symbol": ticker,
        "volume": qty,
        "position": ticket,
        "type": order_type,
        "comment": "Position closed",
        "type_time": mt.ORDER_TIME_GTC,
        "type_filling": mt.ORDER_FILLING_IOC,
    }
    close_order = mt.order_send(request)
    return close_order

In [ ]:
while True:
    
    bought = False
    sold = False
    
    #Retrieve ticket and position type for US30 specifically
    for i in range(len(mt.positions_get())):
        if mt.positions_get()[i]._asdict()['symbol'] == 'US30':
            ticket = mt.positions_get()[i]._asdict()['ticket']
            bought = mt.positions_get()[i]._asdict()['type'] == 0
            sold = mt.positions_get()[i]._asdict()['type'] == 1
    
    #open positions retrieve
    no_positions = len(mt.positions_get()) == 0
    
    #this block calculates time until next candle in second and lets the code sleep for that amt of time
    current_time = datetime.now()
    next_candle_start_time = current_time + timedelta(minutes=1)
    next_candle_start_time = next_candle_start_time.replace(second=0, microsecond=0)
    time_until_next_candle = next_candle_start_time - current_time
    sleep_time = time_until_next_candle.total_seconds()

    if sleep_time > 0:
        time.sleep(sleep_time)
     
    #this block retrieves data from mt5
    rates = mt.copy_rates_from_pos(ticker, mt.TIMEFRAME_M1, 0, 1379)
    if rates is not None and len(rates) > 0:
        data = pd.DataFrame(rates)
        data['time'] = pd.to_datetime(data['time'], unit='s')
        data = data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'tick_volume': 'Vol'})
        data.set_index('time', inplace = True)
        data.drop(columns=['real_volume'], inplace = True)
        data.drop(columns=['spread'], inplace = True)
        print(data)
        
    #taking time of latest row as current time
    hr, m = map(int, data.index.time[1378].strftime('%H, %M').split(','))
    current_time = hr * 100 + m
    
    #all the lines of code are for initializing, scaling and preprocessing of data to input into the trained model
    time_intervals_to_train = 30
    prediction_intervals = 1

    X_input = []
    y = []

    for i in range(time_intervals_to_train, len(data) - prediction_intervals):
        y.append(data.iloc[i + prediction_intervals, 0:4])

    for x in range(time_intervals_to_train, len(data)):
        X_input.append(data.iloc[x - time_intervals_to_train: x, 0:5])

    X_input = np.array(X_input)
    y = np.array(y)

    samples, sequence_length, features = X_input.shape
    X_2d = np.reshape(X_input, (samples * sequence_length, features))

    scaler = MinMaxScaler(feature_range=(0,1))
    X_input = scaler.fit_transform(X_2d)
    y = scaler.fit_transform(y)

    X_input = np.reshape(X_input, (samples, sequence_length, features))
    X = X_input[:-1]

    #Here you can make a condition where after every few hours you can fit all the recent retrieved data to further train the model
    #model.fit(X, y, epochs=15, batch_size=32)

    prediction = model.predict(X_input)
    prediction = scaler.inverse_transform(prediction)
    print('Prediction for next candle in OHLC: ',prediction[-1])
    
    #the strat was to buy if it predicted high 6 or more points above the current candle close
    long_condition = prediction[-1,1] - data.Close[1378]\
    #and sell if it predicted low 6 or more points below the current close
    short_condition = data.Close[1378] - prediction[-1,2] 

    #the time range when this bot trades is 16:30 to 17:00 UTC+2 bcoz mt5 timezone is utc+2
    if 1630 <= current_time <= 1700:
        if not no_positions:
            if bought:
                close_order(ticker, 0.10, ticket, sell) #To close position after every 1m.
            elif sold:
                close_order(ticker, 0.10, ticket, buy)

        if long_condition > 12:
            buy_size = 0.10
            create_order(ticker, buy_size, buy)
            print('bought')

        if short_condition > 12:
            sell_size = 0.10
            print('sold')
    


                         Open      High       Low     Close   Vol
time                                                             
2024-02-27 16:33:00  39024.45  39029.93  38996.04  39005.01   753
2024-02-27 16:34:00  39004.52  39010.50  38968.14  38968.64   794
2024-02-27 16:35:00  38970.13  38970.13  38947.21  38950.20   784
2024-02-27 16:36:00  38950.70  38961.17  38936.75  38953.70   803
2024-02-27 16:37:00  38953.20  38958.15  38932.69  38945.70   761
...                       ...       ...       ...       ...   ...
2024-02-28 16:29:00  38804.38  38809.37  38769.99  38771.99   384
2024-02-28 16:30:00  38772.48  38801.91  38770.00  38775.99  1155
2024-02-28 16:31:00  38775.49  38775.99  38739.10  38761.54   972
2024-02-28 16:32:00  38762.53  38797.43  38746.58  38775.50   834
2024-02-28 16:33:00  38776.50  38777.49  38748.09  38765.54   721

[1379 rows x 5 columns]
43/43 [==============================] - 0s 6ms/step
Prediction for next candle in OHLC:  [38795.305 38809.316 38772.

43/43 [==============================] - 0s 7ms/step
Prediction for next candle in OHLC:  [38768.86  38775.277 38751.52  38766.05 ]
sold
                         Open      High       Low     Close  Vol
time                                                            
2024-02-27 16:42:00  38955.54  38973.80  38953.65  38973.30  846
2024-02-27 16:43:00  38972.78  38974.77  38958.03  38971.57  600
2024-02-27 16:44:00  38971.08  38972.08  38955.94  38961.47  636
2024-02-27 16:45:00  38960.97  38967.44  38952.69  38961.17  677
2024-02-27 16:46:00  38961.19  38968.55  38955.57  38967.56  561
...                       ...       ...       ...       ...  ...
2024-02-28 16:38:00  38761.03  38764.05  38754.61  38757.01  421
2024-02-28 16:39:00  38756.51  38757.04  38743.72  38753.27  434
2024-02-28 16:40:00  38753.77  38768.79  38750.91  38763.80  371
2024-02-28 16:41:00  38763.81  38778.67  38760.81  38774.73  389
2024-02-28 16:42:00  38775.24  38783.05  38767.68  38777.09  437

[1379 rows x 5 co

43/43 [==============================] - 0s 6ms/step
Prediction for next candle in OHLC:  [38788.492 38792.66  38773.016 38788.836]
sold
                         Open      High       Low     Close  Vol
time                                                            
2024-02-27 16:51:00  38953.00  38957.97  38941.06  38944.10  404
2024-02-27 16:52:00  38944.59  38963.99  38943.57  38959.55  424
2024-02-27 16:53:00  38960.05  38967.83  38956.05  38962.70  294
2024-02-27 16:54:00  38963.20  38965.15  38949.09  38959.37  309
2024-02-27 16:55:00  38959.87  38969.91  38957.14  38969.41  309
...                       ...       ...       ...       ...  ...
2024-02-28 16:47:00  38800.59  38802.12  38790.48  38793.00  401
2024-02-28 16:48:00  38793.98  38802.53  38788.49  38802.02  358
2024-02-28 16:49:00  38802.03  38802.55  38789.22  38792.88  289
2024-02-28 16:50:00  38793.87  38795.38  38778.97  38788.37  400
2024-02-28 16:51:00  38788.36  38794.68  38778.53  38781.92  390

[1379 rows x 5 co

bought
                         Open      High       Low     Close  Vol
time                                                            
2024-02-27 16:59:00  38952.14  38963.82  38948.03  38963.82  145
2024-02-27 17:00:00  38962.29  38991.15  38954.94  38960.46  874
2024-02-27 17:01:00  38958.45  38967.38  38954.27  38957.23  597
2024-02-27 17:02:00  38956.74  38975.90  38956.74  38971.40  444
2024-02-27 17:03:00  38971.90  38974.39  38959.39  38965.29  519
...                       ...       ...       ...       ...  ...
2024-02-28 16:55:00  38774.66  38789.32  38772.67  38781.02  329
2024-02-28 16:56:00  38781.04  38783.51  38763.24  38763.74  270
2024-02-28 16:57:00  38763.24  38779.00  38762.76  38772.24  272
2024-02-28 16:58:00  38772.23  38772.59  38766.11  38769.13  201
2024-02-28 16:59:00  38768.13  38774.64  38765.65  38774.63  199

[1379 rows x 5 columns]
43/43 [==============================] - 0s 6ms/step
Prediction for next candle in OHLC:  [38778.184 38784.453 38764.227 38

43/43 [==============================] - 0s 6ms/step
Prediction for next candle in OHLC:  [38789.16  38793.15  38773.695 38789.508]
sold
                         Open      High       Low     Close  Vol
time                                                            
2024-02-27 17:08:00  38932.75  38941.64  38927.73  38939.64  422
2024-02-27 17:09:00  38939.65  38941.14  38930.38  38930.94  428
2024-02-27 17:10:00  38930.44  38936.49  38923.97  38924.96  389
2024-02-27 17:11:00  38924.46  38932.35  38920.99  38932.32  367
2024-02-27 17:12:00  38931.81  38940.18  38927.62  38928.13  341
...                       ...       ...       ...       ...  ...
2024-02-28 17:04:00  38795.30  38797.78  38785.00  38788.02  256
2024-02-28 17:05:00  38789.03  38806.17  38786.50  38806.15  240
2024-02-28 17:06:00  38805.14  38808.10  38800.06  38806.37  253
2024-02-28 17:07:00  38807.86  38811.79  38795.61  38798.08  222
2024-02-28 17:08:00  38798.06  38807.53  38789.97  38802.49  273

[1379 rows x 5 co

...

43/43 [==============================] - 0s 6ms/step
Prediction for next candle in OHLC:  [38911.98  38921.445 38901.625 38912.027]
sold
                         Open      High       Low     Close  Vol
time                                                            
2024-02-27 18:16:00  38919.02  38921.52  38900.67  38909.65  329
2024-02-27 18:17:00  38910.15  38915.18  38905.66  38912.17  290
2024-02-27 18:18:00  38912.67  38914.84  38906.79  38914.34  277
2024-02-27 18:19:00  38914.84  38914.84  38907.91  38910.99  247
2024-02-27 18:20:00  38911.50  38918.43  38907.94  38910.93  279
...                       ...       ...       ...       ...  ...
2024-02-28 18:12:00  38917.18  38919.74  38912.81  38913.32  133
2024-02-28 18:13:00  38913.33  38921.44  38909.36  38921.44  172
2024-02-28 18:14:00  38920.94  38921.93  38916.49  38919.98  158
2024-02-28 18:15:00  38920.47  38922.46  38914.68  38919.19  199
2024-02-28 18:16:00  38919.69  38921.69  38911.73  38912.23  135

[1379 rows x 5 co

43/43 [==============================] - 0s 6ms/step
Prediction for next candle in OHLC:  [38909.785 38919.52  38899.953 38909.63 ]
bought
                         Open      High       Low     Close  Vol
time                                                            
2024-02-27 18:25:00  38920.53  38927.40  38915.07  38925.40  161
2024-02-27 18:26:00  38924.90  38924.90  38917.96  38919.52  162
2024-02-27 18:27:00  38920.02  38921.01  38914.01  38914.52  140
2024-02-27 18:28:00  38914.03  38915.03  38908.08  38909.06  145
2024-02-27 18:29:00  38910.06  38910.29  38903.73  38909.29  232
...                       ...       ...       ...       ...  ...
2024-02-28 18:21:00  38905.96  38915.23  38904.44  38914.74  130
2024-02-28 18:22:00  38914.24  38915.23  38909.75  38911.74  100
2024-02-28 18:23:00  38912.24  38912.74  38902.49  38903.02  115
2024-02-28 18:24:00  38903.52  38904.51  38899.54  38903.52  142
2024-02-28 18:25:00  38903.02  38903.97  38894.48  38895.91  128

[1379 rows x 5 

43/43 [==============================] - 0s 6ms/step
Prediction for next candle in OHLC:  [38897.312 38906.945 38887.414 38896.2  ]
                         Open      High       Low     Close  Vol
time                                                            
2024-02-27 18:34:00  38919.11  38922.82  38915.98  38922.82  166
2024-02-27 18:35:00  38922.79  38922.79  38905.05  38907.56  222
2024-02-27 18:36:00  38908.06  38911.99  38902.92  38902.92  176
2024-02-27 18:37:00  38901.92  38901.92  38883.53  38891.04  361
2024-02-27 18:38:00  38891.03  38897.05  38887.00  38896.05  261
...                       ...       ...       ...       ...  ...
2024-02-28 18:30:00  38901.45  38906.92  38900.54  38901.53  139
2024-02-28 18:31:00  38901.03  38901.03  38886.07  38888.56  142
2024-02-28 18:32:00  38888.06  38888.56  38868.96  38869.46  202
2024-02-28 18:33:00  38869.44  38878.47  38862.96  38877.43  213
2024-02-28 18:34:00  38877.42  38880.32  38871.71  38874.65  126

[1379 rows x 5 columns

43/43 [==============================] - 0s 6ms/step
Prediction for next candle in OHLC:  [38885.61 38894.07 38874.04 38885.24]
                         Open      High       Low     Close  Vol
time                                                            
2024-02-27 18:43:00  38893.68  38897.18  38891.10  38892.07  167
2024-02-27 18:44:00  38891.58  38894.07  38890.09  38891.67  146
2024-02-27 18:45:00  38891.68  38894.14  38884.28  38893.12  238
2024-02-27 18:46:00  38893.62  38898.02  38889.53  38889.53  147
2024-02-27 18:47:00  38888.53  38895.94  38888.52  38894.91  123
...                       ...       ...       ...       ...  ...
2024-02-28 18:39:00  38883.18  38887.96  38879.77  38887.06  160
2024-02-28 18:40:00  38887.07  38887.59  38879.19  38886.10  185
2024-02-28 18:41:00  38886.09  38895.41  38885.59  38889.91  128
2024-02-28 18:42:00  38889.41  38891.55  38886.03  38890.05  148
2024-02-28 18:43:00  38889.56  38892.41  38886.86  38888.45  152

[1379 rows x 5 columns]
43

43/43 [==============================] - 0s 6ms/step
Prediction for next candle in OHLC:  [38897.785 38905.535 38886.043 38898.12 ]
                         Open      High       Low     Close  Vol
time                                                            
2024-02-27 18:52:00  38894.15  38894.65  38885.61  38891.40  128
2024-02-27 18:53:00  38891.37  38892.37  38885.35  38888.90  122
2024-02-27 18:54:00  38888.40  38892.80  38884.31  38888.29  144
2024-02-27 18:55:00  38888.79  38891.28  38887.22  38888.77  140
2024-02-27 18:56:00  38888.78  38897.36  38888.30  38895.38  159
...                       ...       ...       ...       ...  ...
2024-02-28 18:48:00  38901.36  38905.22  38900.79  38903.23  105
2024-02-28 18:49:00  38902.73  38904.72  38899.78  38903.89  112
2024-02-28 18:50:00  38904.39  38913.37  38904.39  38908.87  127
2024-02-28 18:51:00  38909.36  38912.83  38904.84  38906.33  126
2024-02-28 18:52:00  38906.32  38908.80  38902.85  38905.27  123

[1379 rows x 5 columns